<a href="https://colab.research.google.com/github/inefable12/SQP2024/blob/main/Machine_learning_Solubilidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://miro.medium.com/v2/resize:fit:1400/1*GwbGztZOJtP1fNs6584mkQ.gif" width="300" alt="molecula"/>

---
$$\Large \textit{Sociedad Química del Perú | Curso: Inteligencia Artificial para la Escritura Científica}$$

---
<br>

$$\large\textbf{Sesión 3: Machine Learning para la predicción de la solubilidad}$$


$$\textit{Parte Práctica}$$

<br>
<br>



_Jesus Alvarado-Huayhuaz_

## 1. Instalación

In [ ]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
! wget https://raw.githubusercontent.com/dataprofessor/data/master/delaney.csv

In [ ]:
import pandas as pd
sol = pd.read_csv('delaney.csv')
sol

In [ ]:
# Para seleccionar solo una columna, por ejm SMILES
sol.SMILES

In [ ]:
# Para seleccionar solo una instancia de la columna:
sol.SMILES[4]

In [ ]:
# descriptor para 1 instancia
from rdkit import Chem
m = Chem.MolFromSmiles(sol.SMILES[4])
m.GetNumAtoms()

In [ ]:
# para usar todas
from rdkit import Chem
mol_list= []
for element in sol.SMILES:
  mol = Chem.MolFromSmiles(element)
  mol_list.append(mol)

# 2. Calcular descriptores moleculares
Ahora representaremos cada una de las moléculas del conjunto de datos mediante un conjunto de descriptores moleculares que se utilizarán para la construcción de modelos.

1. LogP (coeficiente de partición octanol-agua)
2. MW (peso molecular)
3. RB (Número de enlaces rotativos)
4. AP (proporción aromática = número de átomos aromáticos / número de átomos pesados)

Función personalizada denominada generate() para calcular los 3 descriptores LogP, MW y RB

In [ ]:
import numpy as np
from rdkit.Chem import Descriptors
# Inspiración: https://codeocean.com/explore/capsules?query=tag:data-curation

def generate(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem)
        moldata.append(mol)

    baseData= np.arange(1,1)
    i=0
    for mol in moldata:

        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_MolWt = Descriptors.MolWt(mol)
        desc_NumRotatableBonds = Descriptors.NumRotatableBonds(mol)

        row = np.array([desc_MolLogP,
                        desc_MolWt,
                        desc_NumRotatableBonds])

        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1

    columnNames=["MolLogP","MolWt","NumRotatableBonds"]
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)

    return descriptors

In [ ]:
df = generate(sol.SMILES)
df

In [ ]:
# Sobre el número de átomos aromáticos
SMILES = 'COc1cccc2cc(C(=O)NCCCCN3CCN(c4cccc5nccnc54)CC3)oc21'
m = Chem.MolFromSmiles(SMILES)
aromatic_atoms = [m.GetAtomWithIdx(i).GetIsAromatic() for i in range(m.GetNumAtoms())]
aromatic_atoms
# False: no es un átomo aromático
# True: sí es un átomo aromático

In [ ]:
def AromaticAtoms(m):
  aromatic_atoms = [m.GetAtomWithIdx(i).GetIsAromatic() for i in range(m.GetNumAtoms())]
  aa_count = []
  for i in aromatic_atoms:
    if i==True:
      aa_count.append(1)
  sum_aa_count = sum(aa_count)
  return sum_aa_count

In [ ]:
# Aplicando al ejemplo con "m"
AromaticAtoms(m)

In [ ]:
# Aplicándole a todos, mediante iteración
desc_AromaticAtoms = [AromaticAtoms(element) for element in mol_list]
desc_AromaticAtoms

In [ ]:
# Número de átomos pesados
SMILES = 'COc1cccc2cc(C(=O)NCCCCN3CCN(c4cccc5nccnc54)CC3)oc21'
m = Chem.MolFromSmiles(SMILES)
Descriptors.HeavyAtomCount(m)

In [ ]:
# Aplicándole a todos, mediante iteración
desc_HeavyAtomCount = [Descriptors.HeavyAtomCount(element) for element in mol_list]
desc_HeavyAtomCount

In [ ]:
# Proporción aromática de "m"
SMILES = 'COc1cccc2cc(C(=O)NCCCCN3CCN(c4cccc5nccnc54)CC3)oc21'
m = Chem.MolFromSmiles(SMILES)
AromaticAtoms(m)/Descriptors.HeavyAtomCount(m)

In [ ]:
# Aplicándole a todos, mediante iteración
desc_AromaticProportion = [AromaticAtoms(element)/Descriptors.HeavyAtomCount(element) for element in mol_list]
df_desc_AromaticProportion = pd.DataFrame(desc_AromaticProportion)
df_desc_AromaticProportion

# 3. Preparación del conjunto de datos: Matriz X

In [ ]:
X = pd.concat([df,df_desc_AromaticProportion], axis=1)
X

In [ ]:
# Renombrando las columnas
X = X.rename(columns={'MolLogP':'MolLogP', 'MolWt':'MolWt', 'NumRotatableBonds':'NumRotatableBonds', 0:'AromaticProportion'})
X.columns

In [ ]:
X

# 4. Preparación del conjunto de datos: Matriz Y

In [ ]:
sol.head()

In [ ]:
Y = sol.iloc[:,1]
Y

Separando nuestros datos de entrenamiento y prueba

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
X_train

In [ ]:
Y_train

In [ ]:
X_test

In [ ]:
Y_test

# 5. Modelo de regresión lineal

In [ ]:
# Entrenamiento para RL
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
model = linear_model.LinearRegression()
model.fit(X_train, Y_train)

Predicción y métricas de logS de la data entrenada

In [ ]:
Y_pred_train = model.predict(X_train)
print('Coefficients:', model.coef_)
print('Intercept:', model.intercept_)
print('Mean squared error (MSE): %.2f'
      % mean_squared_error(Y_train, Y_pred_train))
print('Coefficient of determination (R^2): %.2f'
      % r2_score(Y_train, Y_pred_train))

Data de Test

In [ ]:
Y_pred_test = model.predict(X_test)
print('Coefficients:', model.coef_)
print('Intercept:', model.intercept_)
print('Mean squared error (MSE): %.2f'
      % mean_squared_error(Y_test, Y_pred_test))
print('Coefficient of determination (R^2): %.2f'
      % r2_score(Y_test, Y_pred_test))

# ECUACIONES

In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
model = linear_model.LinearRegression()
model.fit(X_train, Y_train)

In [ ]:
yintercept = '%.2f' % model.intercept_
LogP = '%.2f LogP' % model.coef_[0]
MW = '%.4f MW' % model.coef_[1]
RB = '%.4f RB' % model.coef_[2]
AP = '%.2f AP' % model.coef_[3]
print('LogS = ' +
      ' ' +
      yintercept +
      ' + ' +
      LogP +
      ' +' +
      MW +
      ' + ' +
      RB +
      ' + ' +
      AP)

__Full data set__

In [ ]:
# Usando nuestro dataset: x e Y, le llamaremos full
full = linear_model.LinearRegression()
full.fit(X, Y)

In [ ]:
full_pred = model.predict(X)
print('Coefficients:', full.coef_)
print('Intercept:', full.intercept_)
print('Mean squared error (MSE): %.2f'
      % mean_squared_error(Y, full_pred))
print('Coefficient of determination (R^2): %.2f'
      % r2_score(Y, full_pred))

In [ ]:
full_yintercept = '%.2f' % full.intercept_
full_LogP = '%.2f LogP' % full.coef_[0]
full_MW = '%.4f MW' % full.coef_[1]
full_RB = '+ %.4f RB' % full.coef_[2]
full_AP = '%.2f AP' % full.coef_[3]
print('LogS = ' +
      ' ' +
      full_yintercept +
      ' ' +
      full_LogP +
      ' ' +
      full_MW +
      ' ' +
      full_RB +
      ' ' +
      full_AP)

# __Predicción vs Experimental__

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(11,5))

# 1 row, 2 column, plot 1
plt.subplot(1, 2, 1)
plt.scatter(x=Y_train, y=Y_pred_train, c="#7CAE00", alpha=0.3)

z = np.polyfit(Y_train, Y_pred_train, 1)
p = np.poly1d(z)
plt.plot(Y_test,p(Y_test),"#F8766D")

plt.ylabel('Predicted LogS')
plt.xlabel('Experimental LogS')

# 1 row, 2 column, plot 2
plt.subplot(1, 2, 2)
plt.scatter(x=Y_test, y=Y_pred_test, c="#619CFF", alpha=0.3)

z = np.polyfit(Y_test, Y_pred_test, 1)
p = np.poly1d(z)
plt.plot(Y_test,p(Y_test),"#F8766D")

plt.xlabel('Experimental LogS')

#Guardando la gráfica
#plt.savefig('plot_horizontal_logS.png')
#plt.savefig('plot_horizontal_logS.pdf')
plt.show()

Delaney:
__LogS = 0.16 - 0.63 cLogP - 0.0062 MW + 0.066 RB - 0.74 AP__

Pat Walters:
__LogS = 0.26 - 0.74 LogP - 0.0066 MW + 0.0034 RB - 0.42 AP__

TutoBas Train set:
__LogS =  0.26 - 0.73 LogP - 0.0069 MW 0.0163 RB - 0.36 AP__

Full dataset
__LogS =  0.26 - 0.74 LogP - 0.0066 MW + 0.0032 RB - 0.42 AP__